In [47]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import ttest_ind
import plotly.express as px
import db_dtypes
import bigframes.pandas as bpd
from IPython.display import display, HTML
import math
import statsmodels.api as sm
from typing import Union
import logging
import sys
from google.cloud.exceptions import NotFound
from datetime import datetime, timedelta
import time
import os
import json
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google.auth import default  # <-- Make sure to import this
import pandas as pd

def marketing_data(entities):
    
    mkt_data = f"""

    WITH holdout_entities AS (
    SELECT
            entity_id,
            `Release Date` AS release_date
    FROM `logistics-data-storage-staging.long_term_pricing.global_holdout_rollout`
    WHERE `Release Date` < DATE_TRUNC('2025-02-10', WEEK)
    AND entity_id in {entities}
    ),
    orders as (
    SELECT
        dps.entity_id entity_id
        ,dps.dps_customer_id customer_id
        ,COUNT(case when mkt.order_date <= e.release_date then mkt.order_id end) AS orders_pre
        ,COUNT(case when mkt.order_date > e.release_date then mkt.order_id end) AS orders_post
        ,SUM(case when mkt.order_date <= e.release_date then mkt.analytical_profit end) AS analytical_profit_pre
        ,SUM(case when mkt.order_date > e.release_date then mkt.analytical_profit end) AS analytical_profit_post
    FROM `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders` dps
    JOIN holdout_entities AS e
        ON dps.entity_id = e.entity_id
    left join fulfillment-dwh-production.curated_data_shared_mkt.bima_order_profitability mkt
        ON mkt.global_entity_id = dps.entity_id
        AND mkt.order_id = dps.platform_order_code
        AND order_date >= DATE_SUB(release_date, INTERVAL 8 WEEK)
        AND order_date < '2025-02-10'
        AND global_entity_id in {entities}
    WHERE dps.created_date >= DATE_SUB(e.release_date, INTERVAL 8 WEEK) 
        AND dps.created_date < '2025-02-10'
        AND dps.platform_order_code IS NOT NULL
        AND dps.is_own_delivery
        AND dps.is_sent
        AND vendor_vertical_parent IN ('Restaurant','restaurant','restaurants')
        AND dps.entity_id in {entities}
    GROUP BY 1, 2
    ), 
    customer_information AS (
    SELECT
            e.entity_id,
            e.release_date,
            CASE WHEN d.created_date <= e.release_date THEN FALSE ELSE COALESCE(is_customer_holdout, FALSE) END AS is_customer_holdout,
            d.customer_id
    FROM `fulfillment-dwh-production.cl.dps_holdout_users` AS d
    JOIN holdout_entities AS e
            ON d.entity_id = e.entity_id
    LEFT JOIN `fulfillment-dwh-production.cl._bad_dps_logs_ids` bad_ids
        ON d.customer_id = bad_ids.id
    WHERE d.created_date >= '2025-01-01' 
        AND d.created_date < '2025-02-10'
        AND d.customer_id IS NOT NULL
        AND bad_ids.id IS NULL
        AND d.entity_id in {entities}
    GROUP BY 1, 2, 3, 4
    )
    SELECT 
    e.entity_id,
    e.customer_id,
    e.is_customer_holdout,
    o.orders_pre,
    o.orders_post,
    o.analytical_profit_pre,
    o.analytical_profit_post
    FROM customer_information e
    LEFT JOIN orders o
    ON o.customer_id = e.customer_id
    AND o.entity_id = e.entity_id

    """

    return mkt_data

def dps_data():

    dps_data = """

    WITH holdout_entities AS (
    SELECT
            entity_id,
            `Release Date` AS release_date
    FROM `logistics-data-storage-staging.long_term_pricing.global_holdout_rollout`
    WHERE `Release Date` < DATE_TRUNC(CURRENT_DATE(), WEEK)
    AND entity_id not in ('FP_PK','PY_DO','PY_BO', 'FP_TW', 'PY_PY', 'DJ_CZ', 'PY_EC','MJM_AT' ,'PY_PE', 'PY_AR' ,'PY_GT','PY_SV' ,'FP_PH','PY_NI' ,'NP_HU' ,'FP_MM','EF_GR' ,'AP_PA' ,'YS_TR', 'PY_UY' ,'OP_SE' ,'PY_CL' ,'FP_BD' ,'FP_SG' ,'FO_NO' ,'PY_CR', 'FP_LA','PY_HN', 'FP_MY' ,'FP_TH', 'FY_CY', 'PY_VE','PO_FI','TB_QA','TB_OM','TB_KW','TB_JO','TB_IQ','TB_BH','TB_AE','HS_SA')
    ),
    orders as (
    SELECT
        dps.entity_id entity_id
        ,dps.dps_customer_id customer_id
        ,COUNT(case when dps.created_date <= e.release_date then dps.platform_order_code end) AS orders_pre
        ,COUNT(case when dps.created_date > e.release_date then dps.platform_order_code end) AS orders_post
        ,SUM(case when dps.created_date <= e.release_date then dps.fully_loaded_gross_profit_eur end) AS analytical_profit_pre
        ,SUM(case when dps.created_date > e.release_date then dps.fully_loaded_gross_profit_eur end) AS analytical_profit_post
    FROM `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders` dps
    JOIN holdout_entities AS e
        ON dps.entity_id = e.entity_id
    WHERE dps.created_date >= DATE_SUB(e.release_date, INTERVAL 8 WEEK) 
        AND dps.created_date < CURRENT_DATE
        AND dps.platform_order_code IS NOT NULL
        AND dps.is_own_delivery
        AND dps.is_sent
        AND vendor_vertical_parent IN ('Restaurant','restaurant','restaurants')
        AND dps.entity_id not in ('FP_PK','PY_DO','PY_BO', 'FP_TW', 'PY_PY', 'DJ_CZ', 'PY_EC','MJM_AT' ,'PY_PE', 'PY_AR' ,'PY_GT','PY_SV' ,'FP_PH','PY_NI' ,'NP_HU' ,'FP_MM','EF_GR' ,'AP_PA' ,'YS_TR', 'PY_UY' ,'OP_SE' ,'PY_CL' ,'FP_BD' ,'FP_SG' ,'FO_NO' ,'PY_CR', 'FP_LA','PY_HN', 'FP_MY' ,'FP_TH', 'FY_CY', 'PY_VE','PO_FI','TB_QA','TB_OM','TB_KW','TB_JO','TB_IQ','TB_BH','TB_AE','HS_SA')
    GROUP BY 1, 2
    ), 
    customer_information AS (
    SELECT
            e.entity_id,
            e.release_date,
            CASE WHEN d.created_date <= e.release_date THEN FALSE ELSE COALESCE(is_customer_holdout, FALSE) END AS is_customer_holdout,
            d.customer_id
    FROM `fulfillment-dwh-production.cl.dps_holdout_users` AS d
    JOIN holdout_entities AS e
            ON d.entity_id = e.entity_id
    LEFT JOIN `fulfillment-dwh-production.cl._bad_dps_logs_ids` bad_ids
        ON d.customer_id = bad_ids.id
    WHERE d.created_date BETWEEN '2025-01-01' AND CURRENT_DATE()
        AND d.customer_id IS NOT NULL
        AND bad_ids.id IS NULL
    GROUP BY 1, 2, 3, 4
    )
    SELECT 
    e.entity_id,
    e.customer_id,
    e.is_customer_holdout,
    o.orders_pre,
    o.orders_post,
    o.analytical_profit_pre,
    o.analytical_profit_post
    FROM customer_information e
    LEFT JOIN orders o
    ON o.customer_id = e.customer_id
    AND o.entity_id = e.entity_id

    """

    return dps_data


def initialize_bigquery_client(project_id_pass):
   
    # Define your Google Cloud project ID
    project_id = project_id_pass 

    logging.info(f"Initializing BigQuery client for project: {project_id}")

    # Determine credentials path (GitHub Actions vs. Local)
    if os.getenv("GITHUB_ACTIONS"):
        credentials_path = "/tmp/credentials.json" 
    else:
        credentials, project = default()  
        project_id = project if project else project_id 

    if os.getenv("GITHUB_ACTIONS") and not os.path.exists(credentials_path):
        raise FileNotFoundError(f"Credentials file not found at {credentials_path}. Make sure to set up authentication in GitHub Actions.")

    if os.getenv("GITHUB_ACTIONS"):
        with open(credentials_path, "r") as f:
            creds_data = json.load(f)

        credentials = Credentials.from_authorized_user_info(creds_data)

        if credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())

    # Initialize the BigQuery client with explicit credentials and project_id
    try:
        client = bigquery.Client(credentials=credentials, project=project_id)
        logging.info(f"BigQuery client initialized successfully for project: {project_id}")
        return client  # Return the BigQuery client for further use
    except Exception as e:
        logging.error(f"Failed to initialize BigQuery client: {e}")
        raise e


def combined_data(client, mkt, dps):
    
    try:
        mkt_df = client.query(mkt).to_dataframe()
    except Exception as e:
        logging.error(f"Error executing marketing data query: {e}")
        mkt_df = pd.DataFrame() 

    try:
        dps_df = client.query(dps).to_dataframe()
    except Exception as e:
        logging.error(f"Error executing DPS data query: {e}")
        dps_df = pd.DataFrame()

    # Combine the DataFrames
    combined = pd.concat([mkt_df, dps_df], ignore_index=True)

    return combined


In [48]:
## Define the list of entities that have data in marketing tables ## 
## --> rest of the data is extracted from DPS with condition not in the following entities ##

# mkt_entity_id = tuple(set((
#     'FP_PK','PY_DO','PY_BO', 'FP_TW', 
#     'PY_PY', 'DJ_CZ', 'PY_EC','MJM_AT' ,'PY_PE', 'PY_AR',
#     'PY_GT','PY_SV' ,'FP_PH','PY_NI' ,'NP_HU' ,'FP_MM','EF_GR',
#     'AP_PA','YS_TR', 'PY_UY' ,'OP_SE' ,'PY_CL' ,'FP_BD' ,'FP_SG',
#     'FO_NO' ,'PY_CR', 'FP_LA','PY_HN', 'FP_MY' ,'FP_TH', 'FY_CY',
#     'PY_VE','PO_FI','TB_QA','TB_OM','TB_KW','TB_JO','TB_IQ',
#     'TB_BH','TB_AE','HS_SA'
# )))

mkt_entity_id = (
    'FP_PK','PY_DO'
)



project = "logistics-customer-staging"

client = initialize_bigquery_client(project)

test  = client.query(marketing_data(mkt_entity_id)).to_dataframe()

/Users/shazeb.asad/global_pricing/venv_bayesian/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
def fill_nans(df):

    df = df.fillna({
        'orders_pre': 0,
        'orders_post': 0,
        'analytical_profit_pre': 0,
        'analytical_profit_post': 0,
    })

    return df

In [ ]:
def drop_users_with_no_data(df):

    mask1 = (
        (df['orders_pre'] == 0) &
        (df['orders_post'] == 0) &
        (df['analytical_profit_pre'] == 0) &
        (df['analytical_profit_post'] == 0)
    )

    mask2 = (
        (df['orders_pre'].isna()) &
        (df['orders_post'].isna()) &
        (df['analytical_profit_pre'].isna()) &
        (df['analytical_profit_post'].isna())
    ) 

    df = df[~(mask1 | mask2)]
    
    return df

In [ ]:
df = drop_users_with_no_data(raw_data)